# Bài toán

> **Thu thập dữ liệu báo Việt Nam (Vietnamnet)**


Cũng giống như VNExpress - Demo

Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ các trang báo Việt Nam
- Thu thập dữ liệu (bài báo) từ các trang báo Việt Nam để làm dữ liệu cho các bước xử lý sau

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
     {
        "url": "https://vietnamnet.vn/khi-gap-van-xui-hay-nhin-3-...",
        "title": "Khi gặp vận xui, hãy nhìn 3 bức ảnh này sẽ thấy cuộc đời ...",
        "description": "Cuộc đời như thế nào tùy thuộc vào góc nhìn của từng ...",
        "content": "Bức tranh 1: Có 3 hay 4 khúc gỗ?\nCâu trả lời là 3 hoặc 4 khúc gỗ ...",
        "metadata": {
            "cat": "ĐỜI SỐNG",
            "subcat": "GIỚI TRẺ",
            "published_date": 1721274540,
            "author": "Phan Hằng"
        },
    },
    ```


# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [1]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
!apt install -qq chromium-chromedriver

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Install selenium
!pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir -p data

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file data already exists.
Error occurred while processing: data.


In [3]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [4]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [5]:
MAGAZINE_NAME = "vnexpress"
HOME_PAGE = "https://vnexpress.net/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


In [10]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)


In [13]:
# Chọn and click menu buttons

#### Choose menu button
all_menu = driver.find_element(by=By.CLASS_NAME, value="hamburger")
#### CLick on the menu button
all_menu.click()

In [14]:
# Lấy hết tất cả thể loại từ đây
cats = []
EXCLUDING_CATEGORIES = ["Thời sự", "Góc nhìn", "Kinh doanh", "Đời sống", "Pháp luật", "Sức khỏe", "Thế giới", "KHCN", "Thể thao", "Giải trí", "Du lịch", "Giáo dục"]
#### CODE
# row_menu = driver.find_elements(by=By.CLASS_NAME, value="header_submenu-content")
sub_menu = driver.find_element(by=By.CLASS_NAME, value="row-menu")

cat_menus = sub_menu.find_elements(by=By.TAG_NAME, value="a")

for cat_name in cat_menus:
    cat = cat_name.get_attribute("title").strip()
    href = cat_name.get_attribute("href").strip()
    
    # bỏ qua logo-premium, logo-htvn
    if cat not in EXCLUDING_CATEGORIES:
        continue
    else:
        cats.append({"cat_names": cat, "url": href})
####

In [15]:
cats, len(cats)

([{'cat_names': 'Thời sự', 'url': 'https://vnexpress.net/thoi-su'},
  {'cat_names': 'Thế giới', 'url': 'https://vnexpress.net/the-gioi'},
  {'cat_names': 'Kinh doanh', 'url': 'https://vnexpress.net/kinh-doanh'},
  {'cat_names': 'Giải trí', 'url': 'https://vnexpress.net/giai-tri'},
  {'cat_names': 'Pháp luật', 'url': 'https://vnexpress.net/phap-luat'},
  {'cat_names': 'Góc nhìn', 'url': 'https://vnexpress.net/goc-nhin'},
  {'cat_names': 'Sức khỏe', 'url': 'https://vnexpress.net/suc-khoe'},
  {'cat_names': 'Thể thao', 'url': 'https://vnexpress.net/the-thao'},
  {'cat_names': 'Thời sự', 'url': 'https://vnexpress.net/y-kien/thoi-su'},
  {'cat_names': 'Đời sống', 'url': 'https://vnexpress.net/y-kien/doi-song'},
  {'cat_names': 'Giáo dục', 'url': 'https://vnexpress.net/giao-duc'},
  {'cat_names': 'Đời sống', 'url': 'https://vnexpress.net/doi-song'},
  {'cat_names': 'Du lịch', 'url': 'https://vnexpress.net/du-lich'}],
 13)

In [16]:
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt thông số

In [17]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 200 # có thể tăng lên

# Đường dẫn lưu data của vnexpress
DATA_URL_FILE = "data/vnexpress_url.json"

# EXCLUDING_CATEGORIES = ["Giảm nghèo bền vững", "English", "Toàn văn", "Đính chính"] # Exclude

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"
driver = webdriver.Chrome(options=chrome_options)

#### Chạy thử nghiệm



In [96]:
driver = webdriver.Chrome(options=chrome_options)

In [97]:
cats[1]['url']

'https://vnexpress.net/the-gioi'

In [98]:
# Vào thử một url
driver.get(cats[1]['url'])

In [21]:
# Get all titles item

#### CODE
title_news = driver.find_elements(by=By.CLASS_NAME, value="title-news")
####

In [73]:
print(title_news)

[<selenium.webdriver.remote.webelement.WebElement (session="65f371b19b041d694eba27c77f41bbb8", element="f.0B8C68F7DBBE63235DA14B4736E7C754.d.E65F9A9E63E1D6F9B2BDDD8F3D2CD2A9.e.52")>, <selenium.webdriver.remote.webelement.WebElement (session="65f371b19b041d694eba27c77f41bbb8", element="f.0B8C68F7DBBE63235DA14B4736E7C754.d.E65F9A9E63E1D6F9B2BDDD8F3D2CD2A9.e.53")>, <selenium.webdriver.remote.webelement.WebElement (session="65f371b19b041d694eba27c77f41bbb8", element="f.0B8C68F7DBBE63235DA14B4736E7C754.d.E65F9A9E63E1D6F9B2BDDD8F3D2CD2A9.e.54")>, <selenium.webdriver.remote.webelement.WebElement (session="65f371b19b041d694eba27c77f41bbb8", element="f.0B8C68F7DBBE63235DA14B4736E7C754.d.E65F9A9E63E1D6F9B2BDDD8F3D2CD2A9.e.55")>, <selenium.webdriver.remote.webelement.WebElement (session="65f371b19b041d694eba27c77f41bbb8", element="f.0B8C68F7DBBE63235DA14B4736E7C754.d.E65F9A9E63E1D6F9B2BDDD8F3D2CD2A9.e.56")>, <selenium.webdriver.remote.webelement.WebElement (session="65f371b19b041d694eba27c77f41bb

In [22]:
# Lấy đường dẫn

#### CODE
url_new = title_news[0].find_element(by=By.TAG_NAME, value="a").get_attribute("href")
url_new
####

'https://vnexpress.net/tiem-kich-my-lao-xuong-bien-4929616.html'

In [ ]:
# Tìm next page

#### CODE
page_numbers = driver.find_element(by=By.CLASS_NAME, value="next-page")

page_numbers.click()

# next_page = page_numbers.find_elements(by=By.TAG_NAME, value="a")
# next_page = next_page[0].get_attribute("href")
# print(next_page)
####

In [102]:
driver.close()

#### Chạy thật

In [113]:
driver = webdriver.Chrome(options=chrome_options)

In [114]:
EXCLUDING_CATEGORIES = ["Thời sự", "Góc nhìn", "Kinh doanh", "Đời sống", "Pháp luật", "Sức khỏe", "Thế giới", "KHCN", "Thể thao", "Giải trí", "Du lịch", "Giáo dục"]

In [68]:
crawled_urls = {}

In [ ]:
# Global variables for filtering deduplicating urls
crawled_urls = set()

def crawl_each_category_url(driver, category_url):
    """
    Functions cho lấy urls cho từng category sau khi thử nghiệm
    """
    # Get all item-news class item
    all_urls = set()
    url = category_url
    print(url)
    
    #### CODE HERE
    while len(all_urls) < 3:
        driver.get(url)
        parent_paper_url = driver.find_element(by=By.CLASS_NAME, value="list-news-subfolder")
        list_paper = parent_paper_url.find_elements(by=By.CLASS_NAME, value="item-news-common")

        for paper in list_paper[2:]:
            try:
                # crawl paper url
                url_html = paper.find_element(by=By.TAG_NAME, value="a")
                url_new = url_html.get_attribute("href")
                if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls:
                    all_urls.add(url_new)
                    crawled_urls.add(url_new)
                print("url",crawled_urls)
                # crawl avatar paper
                avatar_paper = paper.find_element(by=By.CLASS_NAME, value="thumb-art").find_element(By.CSS_SELECTOR, "picture img").get_attribute("data-src")
                if not avatar_paper:
                    print(f"false in {url_new}")
                else:
                    print("anh",avatar_paper)
                
            except StaleElementReferenceException:
                continue
            except NoSuchElementException:
                print(f"NoSuchElementException at {url}")
                continue
        next_pages = driver.find_element(by=By.CLASS_NAME, value="next-page")
        
        # next_pages = driver.find_element(by=By.CLASS_NAME, value="pagination")
        # url = next_pages.find_elements(by=By.TAG_NAME, value="a")[4].get_attribute("href")
        
    ####

    return all_urls

In [115]:
# Global variables for filtering / deduplicating urls
crawled_urls = set()

TARGET_N = 250  # số bài cần lấy

def crawl_each_category_url(driver, category_url):
    """
    Crawl urls cho từng category: trả về list dict [{paper_url, avatar_url}, ...]
    Dừng khi đủ TARGET_N hoặc không còn trang tiếp theo.
    """
    results = []
    url = category_url
    print(f"[START] category: {category_url}")

    while len(results) < TARGET_N and url:
        driver.get(url)

        try:
            parent_paper_url = driver.find_element(By.CLASS_NAME, "list-news-subfolder")
            list_paper = parent_paper_url.find_elements(By.CLASS_NAME, "item-news-common")
        except NoSuchElementException:
            print(f"[WARN] Không tìm thấy danh sách bài ở: {url}")
            break

        # Bỏ 2 item đầu nếu là spotlight như code trước đó
        for paper in list_paper[2:]:
            if len(results) >= TARGET_N:
                break
            try:
                # paper url
                url_html = paper.find_element(By.TAG_NAME, "a")
                url_new = url_html.get_attribute("href") or ""

                # avatar url (data-src ưu tiên, fallback src)
                img_el = paper.find_element(By.CLASS_NAME, "thumb-art").find_element(By.CSS_SELECTOR, "picture img")
                avatar_paper = img_el.get_attribute("data-src") or img_el.get_attribute("src") or ""

                if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls:
                    crawled_urls.add(url_new)
                    results.append({
                        "paper_url": url_new,
                        "avatar_url": avatar_paper
                    })
            except StaleElementReferenceException:
                continue
            except NoSuchElementException:
                # thiếu ảnh hoặc anchor thì bỏ qua item này
                continue
            except Exception as e:
                print(f"[SKIP] Lỗi không mong muốn khi xử lý item ở {url}: {e}")
                continue

        print(f"[PROGRESS] {len(results)}/{TARGET_N} collected | last page: {url}")

        if len(results) >= TARGET_N:
            break

        # Tìm next page (class = "next-page")
        next_href = None
        try:
            next_pages = driver.find_element(By.CLASS_NAME, "next-page")
            # TH1: chính element có href
            next_href = next_pages.get_attribute("href")
            # TH2: href nằm trong thẻ a con
            if not next_href:
                try:
                    next_href = next_pages.find_element(By.TAG_NAME, "a").get_attribute("href")
                except NoSuchElementException:
                    next_href = None
            # TH3: không có href → thử click (nếu là nút), sau đó lấy current_url
            if not next_href:
                try:
                    driver.execute_script("arguments[0].click();", next_pages)
                    # đợi trang đổi URL một chút (tùy cần)
                    # import time; time.sleep(0.5)
                    next_href = driver.current_url if driver.current_url != url else None
                except Exception:
                    next_href = None
        except NoSuchElementException:
            next_href = None

        if not next_href or next_href == url:
            print("[INFO] Không tìm thấy trang tiếp theo. Dừng.")
            break

        url = next_href  # sang trang kế

    print(f"[DONE] Collected {len(results)} items for category {category_url}")
    return results


In [ ]:
# Global variables for filtering deduplicating urls
crawled_urls = set()

def crawl_each_category_url(driver, category_url):
    """
    Crawl urls cho từng category (bài báo + avatar)
    """
    # Lưu kết quả dạng list of dict
    results = []
    url = category_url
    print(url)

    while len(results) < 3:
        driver.get(url)
        parent_paper_url = driver.find_element(by=By.CLASS_NAME, value="list-news-subfolder")
        list_paper = parent_paper_url.find_elements(by=By.CLASS_NAME, value="item-news-common")

        for paper in list_paper[2:]:
            try:
                # Crawl paper url
                url_html = paper.find_element(by=By.TAG_NAME, value="a")
                url_new = url_html.get_attribute("href")

                # Crawl avatar url
                avatar_paper = paper.find_element(
                    by=By.CLASS_NAME, 
                    value="thumb-art"
                ).find_element(By.CSS_SELECTOR, "picture img").get_attribute("data-src")

                if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls:
                    crawled_urls.add(url_new)
                    results.append({
                        "paper_url": url_new,
                        "avatar_url": avatar_paper
                    })

            except StaleElementReferenceException:
                continue
            except NoSuchElementException:
                print(f"NoSuchElementException at {url}")
                continue

        # Next page
        next_pages = driver.find_element(by=By.ID, value="pagination")
        # url = next_pages.find_elements(by=By.CLASS_NAME, value="btn-page next-page").get_attribute("href")

    return results


In [112]:
_ = crawl_each_category_url(driver, "https://vnexpress.net/thoi-su")


[START] category: https://vnexpress.net/thoi-su
[PROGRESS] 0/250 collected | last page: https://vnexpress.net/thoi-su
[PROGRESS] 0/250 collected | last page: https://vnexpress.net/thoi-su-p2
[PROGRESS] 0/250 collected | last page: https://vnexpress.net/thoi-su-p3
[PROGRESS] 0/250 collected | last page: https://vnexpress.net/thoi-su-p4


KeyboardInterrupt: 

In [116]:
saved_cats = {}

# Thu thập cho từng thể loại
for cat in cats:
    cat_name = cat['cat_names']
    url = cat['url']
    if cat_name in EXCLUDING_CATEGORIES:
        print(f"You are at {cat}.")
        urls = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = list(urls)

with open(DATA_URL_FILE, "w", encoding="utf-8") as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

You are at {'cat_names': 'Thời sự', 'url': 'https://vnexpress.net/thoi-su'}.
[START] category: https://vnexpress.net/thoi-su
[PROGRESS] 10/250 collected | last page: https://vnexpress.net/thoi-su
[PROGRESS] 37/250 collected | last page: https://vnexpress.net/thoi-su-p2
[PROGRESS] 62/250 collected | last page: https://vnexpress.net/thoi-su-p3
[PROGRESS] 88/250 collected | last page: https://vnexpress.net/thoi-su-p4
[PROGRESS] 114/250 collected | last page: https://vnexpress.net/thoi-su-p5
[PROGRESS] 141/250 collected | last page: https://vnexpress.net/thoi-su-p6
[PROGRESS] 167/250 collected | last page: https://vnexpress.net/thoi-su-p7
[PROGRESS] 194/250 collected | last page: https://vnexpress.net/thoi-su-p8
[PROGRESS] 221/250 collected | last page: https://vnexpress.net/thoi-su-p9
[PROGRESS] 247/250 collected | last page: https://vnexpress.net/thoi-su-p10
[PROGRESS] 250/250 collected | last page: https://vnexpress.net/thoi-su-p11
[DONE] Collected 250 items for category https://vnexpre

In [119]:
len(saved_cats)
for cat_name, urls in saved_cats.items():
    print(f"Category: {cat_name}\n")
    if urls:
        print("Sample URL:", urls[0])
    else:
        print(" Không lấy được URL nào.")
    break   

Category: Thời sự

Sample URL: {'paper_url': 'https://vnexpress.net/duong-di-ben-xe-mien-dong-moi-gia-ve-metro-ben-thanh-suoi-tien-ben-xe-mien-dong-4929621.html', 'avatar_url': 'https://i1-vnexpress.vnecdn.net/2025/08/21/bxmPNG-1755743307-6830-1755743363.png?w=500&h=300&q=100&dpr=1&fit=crop&s=Q6SfQHaqNj-UCQsx8UHmBA'}


### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [ ]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vnexpress_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = None # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data\\vnexpress"

!mkdir -p $DATA_FOLDER_OUTPUT

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file data\vnexpress already exists.
Error occurred while processing: data\vnexpress.


In [122]:
FILE_URL_PATH = "data/vnexpress_url.json"  
MAX_ARTICLES_PER_CAT = None
DATA_FOLDER_OUTPUT = os.path.join("data", "vnexpress_url")  
os.makedirs(DATA_FOLDER_OUTPUT, exist_ok=True)

In [153]:
# Đọc file url

with open(FILE_URL_PATH, "r", encoding="utf-8") as fIn:
    url_data = json.load(fIn)

len(url_data)
print(url_data)

{'Thời sự': [{'paper_url': 'https://vnexpress.net/duong-di-ben-xe-mien-dong-moi-gia-ve-metro-ben-thanh-suoi-tien-ben-xe-mien-dong-4929621.html', 'avatar_url': 'https://i1-vnexpress.vnecdn.net/2025/08/21/bxmPNG-1755743307-6830-1755743363.png?w=500&h=300&q=100&dpr=1&fit=crop&s=Q6SfQHaqNj-UCQsx8UHmBA'}, {'paper_url': 'https://vnexpress.net/tien-len-tho-cu-bang-gia-dat-tphcm-2025-bat-dong-san-thua-ke-4929387.html', 'avatar_url': 'https://i1-vnexpress.vnecdn.net/2025/08/20/thocuPNG-1755679443-9812-1755679468.png?w=500&h=300&q=100&dpr=1&fit=crop&s=7of2bmOdvKiA6huysMfKuA'}, {'paper_url': 'https://vnexpress.net/mercedes-benz-club-black-tai-xe-mercedes-danh-nguoi-do-xe-sai-quy-dinh-do-xe-truoc-cua-nha-nguoi-khac-co-bi-phat-khong-nam-dam-cua-tai-xe-mercedes-bi-chu-quan-nhac-nho-4928617.html', 'avatar_url': 'https://i1-vnexpress.vnecdn.net/2025/08/20/geminigeneratedimagehzutzahzut-7008-6687-1755660735.jpg?w=500&h=300&q=100&dpr=1&fit=crop&s=mj2FoOJonrKRFuX2G4WOHA'}, {'paper_url': 'https://vnexpres

In [154]:
for cat_name, articles in url_data.items():
    print(f"\n=== {cat_name} ===")
    for art in articles:
        print(art["paper_url"])


=== Thời sự ===
https://vnexpress.net/duong-di-ben-xe-mien-dong-moi-gia-ve-metro-ben-thanh-suoi-tien-ben-xe-mien-dong-4929621.html
https://vnexpress.net/tien-len-tho-cu-bang-gia-dat-tphcm-2025-bat-dong-san-thua-ke-4929387.html
https://vnexpress.net/mercedes-benz-club-black-tai-xe-mercedes-danh-nguoi-do-xe-sai-quy-dinh-do-xe-truoc-cua-nha-nguoi-khac-co-bi-phat-khong-nam-dam-cua-tai-xe-mercedes-bi-chu-quan-nhac-nho-4928617.html
https://vnexpress.net/chatgpt-com-la-gi-tri-tue-nhan-tao-chatgpt-thoi-nao-vi-dung-ai-4928941.html
https://vnexpress.net/hoc-phi-dai-hoc-2025-hoc-phi-tat-ca-dai-hoc-2025-tang-hoc-phi-dai-hoc-2025-luong-cha-me-khong-theo-noi-muc-tang-hoc-phi-dai-hoc-15-mot-nam-cua-con-4929156.html
https://vnexpress.net/y-kien/tac-gia/qh-2064.html
https://vnexpress.net/vi-sao-cau-thu-bong-chuyen-bong-da-viet-do-buoc-mot-te-4928638.html
https://vnexpress.net/y-kien/tac-gia/nguyen-tuan-linh-2065.html
https://vnexpress.net/ngu-du-giac-la-nhu-the-nao-tre-em-can-ngu-du-giac-gio-vao-hoc-l

#### Chạy thử nghiệm

In [126]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm

SAMPLE_ARTICLE_URLS = [
    # "https://vietnamnet.vn/bat-giam-nguyen-thu-truong-bo-tai-nguyen-va-moi-truong-nguyen-linh-ngoc-2304574.html",
    # "https://vietnamnet.vn/ky-su-cong-nghe-ve-que-nuoi-oc-thu-nhe-lai-nua-ty-moi-nam-2304196.html",
    # "https://vietnamnet.vn/nsnd-thanh-hoa-va-nsnd-le-dung-hai-nguoi-thay-dac-biet-cua-nsut-viet-hoan-2304214.html",
    # #author in last
    # "https://vietnamnet.vn/doi-giay-cu-mon-giua-gia-lanh-chau-au-cua-tong-bi-thu-nguyen-phu-trong-2303988.html",
    # #class maincontent, table ignore
    "https://vnexpress.net/ca-nang-tien-quy-hiem-chet-o-bai-bien-con-dao-4929153.html"
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[0]

In [127]:
driver.get(SAMPLE_ARTICLE_URL)

In [122]:
# Tìm kiếm title

#### CODE HERE
driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text
####


'Cá nàng tiên quý hiếm chết ở bãi biển Côn Đảo'

In [123]:
# Tìm kiếm description

#### CODE HERE
driver.find_element(by=By.CLASS_NAME, value="description").text
####


'TP HCMCá nàng tiên nặng gần 300 kg mắc cạn ở bãi biển Côn Đảo, được người dân và cơ quan chức năng nỗ lực cứu hộ nhưng không thành công.'

In [124]:
# Thu thập thể loại

#### CODE HERE
lis_cat = []
sub_cat = driver.find_element(by=By.CLASS_NAME, value="breadcrumb")
# sub_cat = driver.find_elements(By.CSS_SELECTOR, "a[data-medium^='Menu-']")
# lis_cats = sub_cat.find_element(by=By.CSS_SELECTOR, value="ul").find_elements(by=By.TAG_NAME, value="a")
lis_cats = sub_cat.find_elements(by = By.TAG_NAME,value="a")
for cat in lis_cats:
    category = cat.get_attribute("title")
    if len(category):
        lis_cat.append(category)
        
lis_cat
####

['Thời sự']

In [103]:
#Thu thập ngày
#### CODE HERE
publisd_date = driver.find_element(by=By.CLASS_NAME, value="date").text
publisd_date
####


'Thứ tư, 20/8/2025, 10:12 (GMT+7)'

In [105]:
# Tìm kiếm contents
#### CODE HERE
article = driver.find_element(by=By.CLASS_NAME, value="fck_detail ")
children = article.find_elements(by=By.XPATH, value="./*")
####

In [106]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

article = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "article.fck_detail"))
)

paras = [p.text.strip() for p in article.find_elements(By.CSS_SELECTOR, "p.Normal") if p.text.strip()]

contents = []
author = []

if paras:
    contents = paras[:-1]   
    author = [paras[-1]]    

print( contents)
print(author)

['Con vật mắc cạn dài khoảng 2 m được người dân phát hiện trong tình trạng kiệt sức ở khu vực bãi biển đối diện với trại Phú Hải, chiều 19/8.', 'Một số người hắt nước làm mát cho cá và báo cơ quan chức năng. Nhiều người sau đó tham gia cứu hộ nhưng không thể cứu được. Xác cá sau đó được đưa lên xe chở đi chôn cất.', 'Cá nàng tiên tên khoa học Dugong dugon (còn gọi bò biển, cá cúi), là loài thú ăn cỏ, sống ở vùng nước nông ven bờ Ấn Độ Dương - Thái Bình Dương. Con trưởng thành dài 2,4- 2,7 m, có thể nặng đến 400 kg, tuổi thọ tới 70 năm.', 'Tại Việt Nam, cá phân bố nhiều nơi nhưng nay chỉ còn ghi nhận ở Côn Đảo và Phú Quốc. Loài này nằm trong Sách đỏ IUCN, mức "dễ bị tổn thương", thuộc nhóm IB loài nguy cấp, quý hiếm, nghiêm cấm khai thác.']
['Trường Hà']


In [ ]:
# # Thu thập contents
# contents = []
# author = "Unknown"
# is_slide_show = False

# for idx, child in enumerate(children):
#     text = child.text.strip()
#     if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph

#         #### CODE HERE
#         contents.append(text)
#         ####

#     elif child.tag_name == "figure" :
#         ## If length > 100  --> not a caption, it's next description
#         #### CODE HERE
#         if len(text) > 0:
#             if len(text) <= 100:
#                 contents.append(f"[{text}]")
#             else:
#                 contents.append(text)
#         ####

#     elif child.tag_name == "table": # Do nothing rightnow
#         pass

In [376]:
# Thu thập tác giả
is_author_maybe_last = False
authors = []
try:
    #### CODE HERE
    name = driver.find_element(by=By.CLASS_NAME, value="article-detail-author-wrapper")
    author_names = name.find_elements(by=By.CLASS_NAME, value="name")
    for author_name in author_names:
        author = author_name.find_element(by=By.CSS_SELECTOR, value="a").get_attribute("title")
        authors.append(author)
    ####
except NoSuchElementException:
    is_author_maybe_last = True

# Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
if is_author_maybe_last:
    if len(contents):
        last_content = contents[-1]
        if last_content.find('-') != -1 and last_content.find('-') <= 30:
            authors = last_content[:(last_content.find('-')-1)]
        else:
            authors = "Unknown"

In [125]:
driver.close()

#### Chạy thật

In [138]:
driver = webdriver.Chrome(options=chrome_options)

In [139]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    #### CODE HERE
    title = driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text
    ####

    # Thu thập description
    #### CODE HERE
    description = driver.find_element(by=By.CLASS_NAME, value="description").text
    ####

    # Thu thập thể loại
    #### CODE HERE
    lis_cat = []
    sub_cat = driver.find_element(by=By.CLASS_NAME, value="breadcrumb")
    # sub_cat = driver.find_elements(By.CSS_SELECTOR, "a[data-medium^='Menu-']")
    # lis_cats = sub_cat.find_element(by=By.CSS_SELECTOR, value="ul").find_elements(by=By.TAG_NAME, value="a")
    lis_cats = sub_cat.find_elements(by = By.TAG_NAME,value="a")
    for cat in lis_cats:
        category = cat.get_attribute("title")
        if len(category):
            lis_cat.append(category)
            
    lis_cat
    
    main_cat = lis_cat[0]
    subb_cat = lis_cat[1] if len(lis_cat) > 1 and lis_cat[1] is not None else None
    ####


    # Thu thập published date
    #### CODE HERE
    published_date = driver.find_element(by=By.CLASS_NAME, value="date").text
    ####


    # Thu thập content bài báo
    #### CODE HERE
    article = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "article.fck_detail"))
    )

    paras = [p.text.strip() for p in article.find_elements(By.CSS_SELECTOR, "p.Normal") if p.text.strip()]

    contents = []
    author = []

    if paras:
        contents = paras[:-1]   
        author = [paras[-1]]  
    
    # # Locate phần viết content
    # article = driver.find_element(by=By.CLASS_NAME, value="main-content")
    # # Lấy hết các đầu mục con của bài báo
    # children = article.find_elements(by=By.XPATH, value="./*")
    
    # # Thu thập contents
    # contents = []
    # author = "Unknown"
    # is_slide_show = False

    # for idx, child in enumerate(children):
    #     text = child.text.strip()
    #     if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph

    #         #### CODE HERE
    #         contents.append(text)
    #         ####

    #     elif child.tag_name == "figure" :
    #         ## If length > 100  --> not a caption, it's next description
    #         #### CODE HERE
    #         if len(text) > 0:
    #             if len(text) <= 100:
    #                 contents.append(f"[{text}]")
    #             else:
    #                 contents.append(text)
    #         ####

    #     elif child.tag_name == "table": # Do nothing rightnow
    #         pass
    # ####

    # # Thu thập tác giả
    # #### CODE HERE
    # is_author_maybe_last = False
    # authors = []
    # try:
    #     #### CODE HERE
    #     name = driver.find_element(by=By.CLASS_NAME, value="article-detail-author-wrapper")
    #     author_names = name.find_elements(by=By.CLASS_NAME, value="name")
    #     for author_name in author_names:
    #         author = author_name.find_element(by=By.CSS_SELECTOR, value="a").get_attribute("title")
    #         authors.append(author)
    #     ####
    # except NoSuchElementException:
    #     is_author_maybe_last = True

    # # Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
    # if is_author_maybe_last:
    #     if len(contents):
    #         last_content = contents[-1]
    #         if last_content.find('-') != -1 and last_content.find('-') <= 30:
    #             authors = last_content[:(last_content.find('-')-1)]
    #         else:
    #             authors = "Unknown"


    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": subb_cat,
            "published_date": published_date,
            "author": author
        }
    }


In [133]:
get_content_metadata(driver,SAMPLE_ARTICLE_URL)

{'url': 'https://vnexpress.net/ca-nang-tien-quy-hiem-chet-o-bai-bien-con-dao-4929153.html',
 'title': 'Cá nàng tiên quý hiếm chết ở bãi biển Côn Đảo',
 'description': 'TP HCMCá nàng tiên nặng gần 300 kg mắc cạn ở bãi biển Côn Đảo, được người dân và cơ quan chức năng nỗ lực cứu hộ nhưng không thành công.',
 'content': 'Con vật mắc cạn dài khoảng 2 m được người dân phát hiện trong tình trạng kiệt sức ở khu vực bãi biển đối diện với trại Phú Hải, chiều 19/8.\nMột số người hắt nước làm mát cho cá và báo cơ quan chức năng. Nhiều người sau đó tham gia cứu hộ nhưng không thể cứu được. Xác cá sau đó được đưa lên xe chở đi chôn cất.\nCá nàng tiên tên khoa học Dugong dugon (còn gọi bò biển, cá cúi), là loài thú ăn cỏ, sống ở vùng nước nông ven bờ Ấn Độ Dương - Thái Bình Dương. Con trưởng thành dài 2,4- 2,7 m, có thể nặng đến 400 kg, tuổi thọ tới 70 năm.\nTại Việt Nam, cá phân bố nhiều nơi nhưng nay chỉ còn ghi nhận ở Côn Đảo và Phú Quốc. Loài này nằm trong Sách đỏ IUCN, mức "dễ bị tổn thương", t

In [155]:
# driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url["paper_url"]))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue

    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding="utf-8") as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

# Bug will be emagazines and videos, we ignore them as well

Thu thập dữ liệu thể loại Thời sự ..
Bug at url: {'paper_url': 'https://vnexpress.net/y-kien/tac-gia/qh-2064.html', 'avatar_url': 'https://i1-vnexpress.vnecdn.net/2025/08/18/caulongpng-1755508976-17555089-6592-7532-1755509818.png?w=500&h=300&q=100&dpr=1&fit=crop&s=GHfXuZiSPC7HvVXnSgJkcg'}, with NoSuchElementException
Bug at url: {'paper_url': 'https://vnexpress.net/y-kien/tac-gia/nguyen-tuan-linh-2065.html', 'avatar_url': 'https://i1-vnexpress.vnecdn.net/2025/08/18/tixung4-1755490473-1755490510-4276-1755490601.jpg?w=500&h=300&q=100&dpr=1&fit=crop&s=vDDv4K6ijf-XGGbleoexkg'}, with NoSuchElementException
Thu thập dữ liệu thể loại Thế giới ..
Thu thập dữ liệu thể loại Kinh doanh ..
Bug at url: {'paper_url': 'https://vnexpress.net/tu-ngheo-doi-den-cuong-quoc-luong-thuc-4928390.html', 'avatar_url': 'https://i1-vnexpress.vnecdn.net/2025/08/20/top2-1755620034-6604-175562128-3235-5459-1755645248.png?w=240&h=144&q=100&dpr=1&fit=crop&s=xnMhw9Qg4HncVbJ7Yi6VwA'}, with NoSuchElementException
Bug at 

In [165]:
import json
import os

# Đường dẫn tới folder chứa file JSON
folder_path = "data/vnexpress_url"

# Ví dụ đọc file "đời-sống.json"
file_path = "data/vnexpress_url/sức-khỏe.json"
# file_path = os.path.join(folder_path, file_name)

# Đọc file JSON
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"File có {len(data)} bài.")

FileNotFoundError: [Errno 2] No such file or directory: 'data/vnexpress_url/sức-khỏe.json'

In [393]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue

    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding="utf-8") as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

# Bug will be emagazines and videos, we ignore them as well

Thu thập dữ liệu thể loại Chính trị ..
Thu thập dữ liệu thể loại Thời sự ..
Thu thập dữ liệu thể loại Kinh doanh ..
Thu thập dữ liệu thể loại Thể thao ..
Thu thập dữ liệu thể loại Thế giới ..
Thu thập dữ liệu thể loại Giáo dục ..
Thu thập dữ liệu thể loại Giải trí ..
Bug at url: https://vietnamnet.vn/my-nhan-cao-1-72m-hoc-thac-si-kiem-tien-tu-nam-16-tuoi-thi-miss-grand-vietnam-2306665.html, with NoSuchElementException
Thu thập dữ liệu thể loại Văn hóa ..
Thu thập dữ liệu thể loại Đời sống ..
Thu thập dữ liệu thể loại Sức khỏe ..
Thu thập dữ liệu thể loại Thông tin và Truyền thông ..
Thu thập dữ liệu thể loại Pháp luật ..
Thu thập dữ liệu thể loại Ô tô xe máy ..
Thu thập dữ liệu thể loại Bất động sản ..
Thu thập dữ liệu thể loại Du lịch ..
Thu thập dữ liệu thể loại Bạn đọc ..
Thu thập dữ liệu thể loại Tuần Việt Nam ..
Thu thập dữ liệu thể loại Công nghiệp hỗ trợ ..
Thu thập dữ liệu thể loại Bảo vệ người tiêu dùng ..
Thu thập dữ liệu thể loại Thị trường tiêu dùng ..
Thu thập dữ liệu thể 

In [394]:
# Xem 1 sample
cat_data[-1]

{'url': 'https://vietnamnet.vn/bac-lieu-phat-huy-quyen-lam-chu-cua-nguoi-dan-trong-xay-dung-nong-thon-moi-2305220.html',
 'title': 'Bạc Liêu: Phát huy quyền làm chủ của người dân trong xây dựng nông thôn mới',
 'description': 'Tỉnh Bạc Liêu luôn chú trọng phát huy quyền làm chủ của nhân dân, lấy ý kiến về sự hài lòng của người dân làm thước đo trong xây dựng nông thôn mới.',
 'content': 'Với phương châm xây dựng nông thôn mới nâng cao, nông thôn mới kiểu mẫu phải đảm bảo thực chất, không chạy theo thành tích, tỉnh Bạc Liêu luôn chú trọng phát huy quyền làm chủ của nhân dân, lấy ý kiến về sự hài lòng của người dân làm thước đo trong xây dựng nông thôn mới.\nTheo đó, Ban Thường trực Ủy ban Mặt trận Tổ quốc Việt Nam các huyện, thị xã, thành phố trong tỉnh đã chủ trì, phối hợp với các tổ chức chính trị - xã hội, các xã để tiến hành phát phiếu lấy ý kiến sự hài lòng của người dân trên địa bàn về kết quả xây dựng xã đạt chuẩn nông thôn mới nâng cao, nông thôn mới kiểu mẫu.\nThông qua việc lấ

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set to your folder
FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
!cp -r data $FOLDER_SAVED_GOOGLE_COLAB